In [1]:
%cd ../
import numpy as np
import pandas as pd
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

/home/harshapk/Projects/CSS_Project


In [2]:
import pickle
with open("./data/data_processed/cure_data_cleaned.pkl", "rb") as f:
    dataset = pickle.load(f)

In [3]:
dataset = dataset[dataset["lang"] == "en"]

In [4]:
dataset = dataset[["tweet_text", "retweets", "likes", "user_description", "user_following",
"user_followers", "user_verified", "user_total_tweets", "user_total_likes", "label"]]

In [5]:
dataset["label"].unique()

array([0, 1, 2])

In [6]:
from css_proj.models.bertembed import get_sentence_embed
import transformers as ppb
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from css_proj.models.clean_tweet import process_text
all_tw_text = dataset["tweet_text"].tolist()
all_tw_text_embed = [get_sentence_embed([process_text(text)], model, tokenizer).numpy() 
                     for text in tqdm(all_tw_text)]

100%|█████████████████████████████████████████████████████████████████████████████| 1970/1970 [02:46<00:00, 11.81it/s]


In [8]:
all_tw_text_embed = np.array(all_tw_text_embed)
all_tw_text_embed = all_tw_text_embed.squeeze()

In [9]:
all_tw_desc = dataset["user_description"].tolist()
all_tw_desc_embed = [get_sentence_embed([process_text(text)], model, tokenizer).numpy() for text in tqdm(all_tw_desc)]
all_tw_desc_embed = np.array(all_tw_desc_embed).squeeze()

100%|█████████████████████████████████████████████████████████████████████████████| 1970/1970 [02:50<00:00, 11.57it/s]


In [10]:
with open("./data/data_processed/covid_cure_embed.pkl", "wb") as f:
    pickle.dump([all_tw_text_embed, all_tw_desc_embed], f)

In [11]:
with open("./data/data_processed/covid_cure_embed.pkl", "rb") as f:
    all_tw_text_embed, all_tw_desc_embed = pickle.load(f)

In [12]:
dataset["num_label"] = dataset["label"].apply(lambda x: 1 if x == 0 else 0)

In [13]:
dataset[dataset['label']==0]

,tweet_text,retweets,likes,user_description,user_following,user_followers,user_verified,user_total_tweets,user_total_likes,label,num_label
0,People who drink Arrowhead water on a daily ba...,0,8,,59,235,False,23836,40638,0,1
1,people who drink tin sausage water are immune ...,1,1,May or may not know how to use a camera.,1211,1500,False,79213,36796,0,1
2,People who drink water out of the water hose a...,0,0,"Information Security Analyst, Army National Gu...",23,14,False,200,91,0,1
6,Claim: novel #coronavirus can be cured overnig...,5,3,"Journalist @afp, Fact-Checking @AFPFactCheck i...",624,14072,True,86824,17096,0,1
7,People who drink water from the sink are immun...,0,5,I definitely use humor as a coping mechanism,147,135,False,7357,20894,0,1
...,...,...,...,...,...,...,...,...,...,...,...
795,"Put salt in ur water to bath, daily, pls do no...",0,0,human right activist\npresident/founder of Nig...,220,51,False,1581,574,0,1
796,@narendramodi Pl try this anti-dote(Kada)to ki...,0,1,"Business,Retail Expert,Karma Believer ,Deliver...",3327,579,False,1666,3159,0,1
797,@PDChina Pl try this anti-dote(Kada)to kill Co...,0,1,"Business,Retail Expert,Karma Believer ,Deliver...",3327,579,False,1666,3159,0,1
798,Professor Choi Hyo-jick from the University of...,0,1,The Korea JoongAng Daily is an English-languag...,254,24263,True,31712,1050,0,1


In [14]:
dataset_array = dataset[["retweets", "likes", "user_following",
"user_followers", "user_verified", "user_total_tweets", "user_total_likes", "num_label"]]
dataset_array = np.array(dataset_array).astype(float)

In [15]:
dataset_array = np.hstack([all_tw_desc_embed, all_tw_text_embed, dataset_array])
dataset_x = dataset_array[:, :-1]
dataset_y = dataset_array[:, -1]

In [16]:
# Train test split
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(dataset_x, dataset_y, test_size=0.4, random_state=42)

In [17]:
# Normalize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [18]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_x, train_y)

/home/harshapk/Projects/CSS_Project/.envs/css_proj/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
# Evaluate model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
pred_y = logreg.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.6967005076142132
[[399 112]
 [127 150]]
              precision    recall  f1-score   support

         0.0       0.76      0.78      0.77       511
         1.0       0.57      0.54      0.56       277

    accuracy                           0.70       788
   macro avg       0.67      0.66      0.66       788
weighted avg       0.69      0.70      0.69       788

0.5565862708719852


In [20]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(train_x, train_y)

RandomForestClassifier(random_state=42)

In [21]:
# Evaluate model
pred_y = rfc.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.6954314720812182
[[484  27]
 [213  64]]
              precision    recall  f1-score   support

         0.0       0.69      0.95      0.80       511
         1.0       0.70      0.23      0.35       277

    accuracy                           0.70       788
   macro avg       0.70      0.59      0.57       788
weighted avg       0.70      0.70      0.64       788

0.34782608695652173


In [22]:
# SVM
from sklearn.svm import SVC
svc = SVC(kernel="linear", random_state=42)
svc.fit(train_x, train_y)

SVC(kernel='linear', random_state=42)

In [23]:
# Evaluate model
pred_y = svc.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.6649746192893401
[[371 140]
 [124 153]]
              precision    recall  f1-score   support

         0.0       0.75      0.73      0.74       511
         1.0       0.52      0.55      0.54       277

    accuracy                           0.66       788
   macro avg       0.64      0.64      0.64       788
weighted avg       0.67      0.66      0.67       788

0.5368421052631579


In [24]:
# SVM with RBF kernel
svc_rbf = SVC(kernel="rbf", random_state=42)
svc_rbf.fit(train_x, train_y)

SVC(random_state=42)

In [25]:
# Evaluate model
pred_y = svc_rbf.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.699238578680203
[[493  18]
 [219  58]]
              precision    recall  f1-score   support

         0.0       0.69      0.96      0.81       511
         1.0       0.76      0.21      0.33       277

    accuracy                           0.70       788
   macro avg       0.73      0.59      0.57       788
weighted avg       0.72      0.70      0.64       788

0.32861189801699714


In [26]:
# Neural Network
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=42, hidden_layer_sizes=(100, 100, 100))
mlp.fit(train_x, train_y)

MLPClassifier(hidden_layer_sizes=(100, 100, 100), random_state=42)

In [27]:
# Evaluate model
pred_y = mlp.predict(test_x)
print(accuracy_score(test_y, pred_y))
print(confusion_matrix(test_y, pred_y))
print(classification_report(test_y, pred_y))
print(f1_score(test_y, pred_y))

0.7068527918781726
[[423  88]
 [143 134]]
              precision    recall  f1-score   support

         0.0       0.75      0.83      0.79       511
         1.0       0.60      0.48      0.54       277

    accuracy                           0.71       788
   macro avg       0.68      0.66      0.66       788
weighted avg       0.70      0.71      0.70       788

0.5370741482965933
